Część 0


In [64]:
import numpy as np
import matplotlib.pyplot as plt
vec_a1 = 1
vec_a2 = 1
vec_a3 = 1
n = 1000
U = np.random.rand(n)
X = np.where(U < 0.5,
             -1 + np.sqrt(2 * U),
             1 - np.sqrt(2 * (1 - U)))

desired_var = 0.1
current_var = np.var(X)
scale_factor = np.sqrt(desired_var / current_var)
noise = np.random.normal(0, np.sqrt(desired_var), n)

#noise = X * scale_factor
rows = []
uk1 = 0
uk2 = 0
y = []
for i in range(n):
    
    value = vec_a1 * U[i] + vec_a2 * uk1 + vec_a3 * uk2 + noise[i]
    y.append(value)
    rows.append([U[i], uk1, uk2])
    uk2 = uk1
    uk1 = U[i]

y = np.array(y).reshape(-1, 1)
rows = np.array(rows)
rows_T = rows.T
fi = np.linalg.inv(rows_T @ rows) @ rows_T @ y

print(fi)

[[0.98844213]
 [0.98950441]
 [1.01723883]]
